# 5. Language Modelling


https://huggingface.co/docs/transformers/task_summary#language-modeling 

https://huggingface.co/models?filter=pl


To properly create pipelines I had to modify "env/lib/python3.9/site-packages/transformers/modeling_tf_utils.py":

```
...
from huggingface_hub import Repository, list_repo_files
# from keras.saving.hdf5_format import save_attributes_to_hdf5_group
from transformers.utils.hub import convert_file_size_to_int, get_checkpoint_shard_files
...
```

The keras module had to be disabled.

In [1]:
from transformers import pipeline
import torch

/Users/filipkatulski/Library/CloudStorage/OneDrive-Personal/Studia/13 SEMESTR/Przetwarzanie Języka Naturalnego/Labki/natural-language-processing/5-lm/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-01 13:45:40.082671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
unmasker_bert = pipeline("fill-mask", model="bert-base-multilingual-cased") ## pobierze sobie model samemu, kazdy ok. 1GB


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
unmasker_distilbert = pipeline("fill-mask", model="distilbert-base-multilingual-cased")

In [4]:
unmasker_mt5_small = pipeline("fill-mask", model="google/mt5-small")

Downloading: 100%|██████████| 1.20G/1.20G [27:01<00:00, 741kB/s]    


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [5]:
unmasker_roberta = pipeline("fill-mask", model="xlm-roberta-base")

Downloading: 100%|██████████| 1.12G/1.12G [14:15<00:00, 1.30MB/s]  
Downloading: 100%|██████████| 5.07M/5.07M [00:04<00:00, 1.15MB/s]
Downloading: 100%|██████████| 9.10M/9.10M [00:05<00:00, 1.57MB/s]


In [18]:
unmasker_facebook = pipeline("fill-mask", model="facebook/m2m100_418M")

Downloading: 100%|██████████| 908/908 [00:00<00:00, 246kB/s]
Downloading: 100%|██████████| 1.94G/1.94G [35:50<00:00, 900kB/s]   
Downloading: 100%|██████████| 272/272 [00:00<00:00, 79.2kB/s]


ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

In [7]:
import pandas as pd

In [21]:
def test_models(s):
    index = pd.MultiIndex.from_tuples([
        ('bert', 'score'),
        ('bert', 'token'),
        ('bert', 'sentence'),

        ('distilbert', 'score'),
        ('distilbert', 'token'),
        ('distilbert', 'sentence'),

        ('roberta', 'score'),
        ('roberta', 'token'),
        ('roberta', 'sentence'),
        
    ], names=['model', 'field'])

    df_bert = pd.DataFrame(
        [(entry['score'], entry['token_str'], entry['sequence']) for entry in unmasker_bert(s)],
        columns=['score', 'token', 'sequence']
    )
    df_distilbert = pd.DataFrame(
        [(entry['score'], entry['token_str'], entry['sequence']) for entry in unmasker_distilbert(s)],
        columns=['score', 'token', 'sequence']
    )
    df_roberta = pd.DataFrame(
        [(entry['score'], entry['token_str'], entry['sequence']) for entry in unmasker_roberta(s.replace('[MASK]', '<mask>'))],
        columns=['score', 'token', 'sequence']
    )
    df = pd.concat([df_bert, df_distilbert, df_roberta], axis=1)
    df.columns = index
    return df

## Mianownik - kto? co?

In [24]:
test_models("Julia to najlepsza [MASK].")

model      bert                                       distilbert          \
field     score    token                     sentence      score   token   
0      0.276868    Julia    Julia to najlepsza Julia.   0.231084       ₖ   
1      0.100446  aktorka  Julia to najlepsza aktorka.   0.088533   write   
2      0.027406    córka    Julia to najlepsza córka.   0.058922   osoba   
3      0.008817   muzyka   Julia to najlepsza muzyka.   0.022007    tuổi   
4      0.007136     żona     Julia to najlepsza żona.   0.014179  kobiet   

model                               roberta                                  
field                    sentence     score token                  sentence  
0           Julia to najlepsza ₖ.  0.078439     .      Julia to najlepsza..  
1       Julia to najlepsza write.  0.024084     s      Julia to najlepszas.  
2       Julia to najlepsza osoba.  0.020558     !      Julia to najlepsza!.  
3        Julia to najlepsza tuổi.  0.014358  girl  Julia to najlepsza girl.  
4      Julia to najlepsza kobiet.  0.010458     e      Julia to najlepszae.

In [25]:
test_models("Julia is the best [MASK].")

model      bert                                    distilbert            \
field     score   token                   sentence      score     token   
0      0.194280    girl    Julia is the best girl.   0.146940    friend   
1      0.076912  sister  Julia is the best sister.   0.096350      girl   
2      0.043963  friend  Julia is the best friend.   0.046619   actress   
3      0.031303      of      Julia is the best of.   0.043551     child   
4      0.023357   known   Julia is the best known.   0.022676  daughter   

model                                roberta           \
field                     sentence     score    token   
0        Julia is the best friend.  0.177938   friend   
1          Julia is the best girl.  0.080004  actress   
2       Julia is the best actress.  0.064894     girl   
3         Julia is the best child.  0.045308        .   
4      Julia is the best daughter.  0.033313    woman   

model                              
field                    sentence  
0       Julia is the best friend.  
1      Julia is the best actress.  
2         Julia is the best girl.  
3             Julia is the best..  
4        Julia is the best woman.

In [27]:
test_models("Jowisz to największa [MASK].")

model      bert                                       distilbert             \
field     score   token                      sentence      score      token   
0      0.051457   część   Jowisz to największa część.   0.060419     rounds   
1      0.039043   sonda   Jowisz to największa sonda.   0.020742      grupa   
2      0.037867      ta      Jowisz to największa ta.   0.017750    gatunek   
3      0.037798   grupa   Jowisz to największa grupa.   0.016106  jednostka   
4      0.035395  planet  Jowisz to największa planet.   0.016092      droga   

model                                    roberta          \
field                         sentence     score   token   
0         Jowisz to największa rounds.  0.116505  miłość   
1          Jowisz to największa grupa.  0.031983  liczba   
2        Jowisz to największa gatunek.  0.016257       .   
3      Jowisz to największa jednostka.  0.015509     gra   
4          Jowisz to największa droga.  0.013595    dama   

model                                
field                      sentence  
0      Jowisz to największa miłość.  
1      Jowisz to największa liczba.  
2            Jowisz to największa..  
3         Jowisz to największa gra.  
4        Jowisz to największa dama.

In [28]:
test_models("Jupiter is the biggest [MASK].")

model      bert                                           distilbert  \
field     score    token                         sentence      score   
0      0.359052  Jupiter  Jupiter is the biggest Jupiter.   0.570830   
1      0.171056   planet   Jupiter is the biggest planet.   0.123706   
2      0.045322     star     Jupiter is the biggest star.   0.022475   
3      0.022692   Saturn   Jupiter is the biggest Saturn.   0.016679   
4      0.021285   object   Jupiter is the biggest object.   0.014521   

model                                                 roberta            \
field      token                           sentence     score     token   
0         planet     Jupiter is the biggest planet.  0.515526    planet   
1           moon       Jupiter is the biggest moon.  0.098849      star   
2         object     Jupiter is the biggest object.  0.035819  asteroid   
3      satellite  Jupiter is the biggest satellite.  0.031860    Planet   
4            Sun        Jupiter is the biggest Sun.  0.026420     angel   

model                                    
field                          sentence  
0        Jupiter is the biggest planet.  
1          Jupiter is the biggest star.  
2      Jupiter is the biggest asteroid.  
3        Jupiter is the biggest Planet.  
4         Jupiter is the biggest angel.

## Dopełniacz - kogo? czego?

In [40]:
test_models("Krowa jest matką [MASK].")

model      bert                             distilbert              \
field     score token              sentence      score       token   
0      0.108822    pt  Krowa jest matką pt.   0.248806           ♠   
1      0.052409     p   Krowa jest matką p.   0.077266       Marii   
2      0.037285    ks  Krowa jest matką ks.   0.050091           ქ   
3      0.031200     r   Krowa jest matką r.   0.021748  Aleksandra   
4      0.030302     K   Krowa jest matką K.   0.017931  Stanisława   

model                                 roberta           \
field                      sentence     score    token   
0               Krowa jest matką ♠.  0.092401        .   
1           Krowa jest matką Marii.  0.074847    dwóch   
2               Krowa jest matką ქ.  0.049575        2   
3      Krowa jest matką Aleksandra.  0.043125  dziecka   
4      Krowa jest matką Stanisława.  0.035836        3   

model                             
field                   sentence  
0             Krowa jest matką..  
1        Krowa jest matką dwóch.  
2            Krowa jest matką 2.  
3      Krowa jest matką dziecka.  
4            Krowa jest matką 3.

In [51]:
test_models("A cow is a mother of [MASK].")

model      bert                                         distilbert          \
field     score    token                       sentence      score   token   
0      0.218452      God      A cow is a mother of God.   0.033958    food   
1      0.082331  science  A cow is a mother of science.   0.026665    milk   
2      0.019501        a        A cow is a mother of a.   0.022396    meat   
3      0.014830     food     A cow is a mother of food.   0.020774  cattle   
4      0.014816      law      A cow is a mother of law.   0.019464    fish   

model                                 roberta         \
field                      sentence     score  token   
0        A cow is a mother of food.  0.250205    two   
1        A cow is a mother of milk.  0.123114      a   
2        A cow is a mother of meat.  0.071799  three   
3      A cow is a mother of cattle.  0.051731   four   
4        A cow is a mother of fish.  0.025352      2   

model                               
field                     sentence  
0        A cow is a mother of two.  
1          A cow is a mother of a.  
2      A cow is a mother of three.  
3       A cow is a mother of four.  
4          A cow is a mother of 2.

## Biernik - kogo? co?

In [42]:
test_models("Naturalnym satelitą Ziemi jest [MASK].")

model      bert                                               distilbert  \
field     score  token                               sentence      score   
0      0.267922  Ziemi  Naturalnym satelitą Ziemi jest Ziemi.   0.056153   
1      0.144967   Mars   Naturalnym satelitą Ziemi jest Mars.   0.037584   
2      0.030142    lat    Naturalnym satelitą Ziemi jest lat.   0.036803   
3      0.017516     np     Naturalnym satelitą Ziemi jest np.   0.029498   
4      0.015865    nie    Naturalnym satelitą Ziemi jest nie.   0.022555   

model                                                     roberta             \
field    token                                 sentence     score      token   
0         Mars     Naturalnym satelitą Ziemi jest Mars.  0.144999          :   
1        Ariel    Naturalnym satelitą Ziemi jest Ariel.  0.030682  niebieski   
2        Ziemi    Naturalnym satelitą Ziemi jest Ziemi.  0.024862       Mars   
3        Marte    Naturalnym satelitą Ziemi jest Marte.  0.022915       </s>   
4      Jupiter  Naturalnym satelitą Ziemi jest Jupiter.  0.021476     Saturn   

model                                             
field                                   sentence  
0               Naturalnym satelitą Ziemi jest:.  
1      Naturalnym satelitą Ziemi jest niebieski.  
2           Naturalnym satelitą Ziemi jest Mars.  
3                Naturalnym satelitą Ziemi jest.  
4         Naturalnym satelitą Ziemi jest Saturn.

In [52]:
test_models("Earth's natural satelite is [MASK].")

model      bert                                                distilbert  \
field     score    token                              sentence      score   
0      0.204759  Jupiter  Earth's natural satelite is Jupiter.   0.038488   
1      0.167143   Saturn   Earth's natural satelite is Saturn.   0.032494   
2      0.092948    Pluto    Earth's natural satelite is Pluto.   0.026574   
3      0.047514   Uranus   Earth's natural satelite is Uranus.   0.026527   
4      0.029999     Mars     Earth's natural satelite is Mars.   0.022855   

model                                                  roberta           \
field    token                              sentence     score    token   
0      Jupiter  Earth's natural satelite is Jupiter.  0.072247        :   
1        Pluto    Earth's natural satelite is Pluto.  0.067615  Jupiter   
2        Venus    Earth's natural satelite is Venus.  0.046156     Mars   
3       Saturn   Earth's natural satelite is Saturn.  0.033522    Venus   
4      Neptune  Earth's natural satelite is Neptune.  0.024352      ...   

model                                        
field                              sentence  
0             Earth's natural satelite is:.  
1      Earth's natural satelite is Jupiter.  
2         Earth's natural satelite is Mars.  
3        Earth's natural satelite is Venus.  
4           Earth's natural satelite is....

## Celownik - komu? czemu?

In [43]:
test_models("Po wywiadzie prezydent podziękował [MASK].")

model      bert                                                              \
field     score       token                                        sentence   
0      0.269986          go          Po wywiadzie prezydent podziękował go.   
1      0.053810         się         Po wywiadzie prezydent podziękował się.   
2      0.048915  prezydenta  Po wywiadzie prezydent podziękował prezydenta.   
3      0.031474         nie         Po wywiadzie prezydent podziękował nie.   
4      0.023077   prezydent   Po wywiadzie prezydent podziękował prezydent.   

model distilbert                                                      \
field      score   token                                    sentence   
0       0.036610  władze  Po wywiadzie prezydent podziękował władze.   
1       0.026931       裙       Po wywiadzie prezydent podziękował 裙.   
2       0.017560  wojska  Po wywiadzie prezydent podziękował wojska.   
3       0.017452   prace   Po wywiadzie prezydent podziękował prace.   
4       0.017124  udział  Po wywiadzie prezydent podziękował udział.   

model   roberta                                                            
field     score      token                                       sentence  
0      0.239111         za         Po wywiadzie prezydent podziękował za.  
1      0.163919          .           Po wywiadzie prezydent podziękował..  
2      0.162770  wszystkim  Po wywiadzie prezydent podziękował wszystkim.  
3      0.045018          :           Po wywiadzie prezydent podziękował:.  
4      0.019201        ...         Po wywiadzie prezydent podziękował....

In [49]:
test_models("After the interview the President thanked [MASK].")

model      bert                                                                \
field     score      token                                           sentence   
0      0.124308        him     After the interview the President thanked him.   
1      0.069588  President  After the interview the President thanked Pres...   
2      0.065176          N       After the interview the President thanked N.   
3      0.018780        the     After the interview the President thanked the.   
4      0.018348          U       After the interview the President thanked U.   

model distilbert                                                              \
field      score    token                                           sentence   
0       0.055133    Obama   After the interview the President thanked Obama.   
1       0.055130      him     After the interview the President thanked him.   
2       0.040453    Trump   After the interview the President thanked Trump.   
3       0.026717  himself  After the interview the President thanked hims...   
4       0.014321     Bush    After the interview the President thanked Bush.   

model   roberta                                                                
field     score      token                                           sentence  
0      0.118899        the     After the interview the President thanked the.  
1      0.056389        him     After the interview the President thanked him.  
2      0.054782       them    After the interview the President thanked them.  
3      0.054518         me      After the interview the President thanked me.  
4      0.031898  President  After the interview the President thanked Pres...

## Narzędnik - z kim? z czym?

In [45]:
test_models("Michał Anioł jest najbardziej znanym [MASK].")

model      bert                                                  distilbert  \
field     score token                                   sentence      score   
0      0.226140    ks   Michał Anioł jest najbardziej znanym ks.   0.136829   
1      0.156555    św   Michał Anioł jest najbardziej znanym św.   0.095137   
2      0.084220    pt   Michał Anioł jest najbardziej znanym pt.   0.083338   
3      0.032939    np   Michał Anioł jest najbardziej znanym np.   0.037624   
4      0.030891   tzw  Michał Anioł jest najbardziej znanym tzw.   0.035027   

model                                                             roberta  \
field     token                                        sentence     score   
0         aktor     Michał Anioł jest najbardziej znanym aktor.  0.213279   
1       autorem   Michał Anioł jest najbardziej znanym autorem.  0.068188   
2         synem     Michał Anioł jest najbardziej znanym synem.  0.062659   
3      pianista  Michał Anioł jest najbardziej znanym pianista.  0.061316   
4             ஓ         Michał Anioł jest najbardziej znanym ஓ.  0.046595   

model                                                          
field    token                                       sentence  
0            z        Michał Anioł jest najbardziej znanym z.  
1            i        Michał Anioł jest najbardziej znanym i.  
2            .         Michał Anioł jest najbardziej znanym..  
3            w        Michał Anioł jest najbardziej znanym w.  
4      polskim  Michał Anioł jest najbardziej znanym polskim.

In [48]:
test_models("Michael Angelo is the most famous [MASK].")

model      bert                                                         \
field     score     token                                     sentence   
0      0.058573    Angelo    Michael Angelo is the most famous Angelo.   
1      0.047160    singer    Michael Angelo is the most famous singer.   
2      0.040553     actor     Michael Angelo is the most famous actor.   
3      0.014596    player    Michael Angelo is the most famous player.   
4      0.013030  musician  Michael Angelo is the most famous musician.   

model distilbert                                                             \
field      score       token                                       sentence   
0       0.107241  footballer  Michael Angelo is the most famous footballer.   
1       0.093704       actor       Michael Angelo is the most famous actor.   
2       0.078382      singer      Michael Angelo is the most famous singer.   
3       0.070442    composer    Michael Angelo is the most famous composer.   
4       0.051473     painter     Michael Angelo is the most famous painter.   

model   roberta                                                       
field     score    token                                    sentence  
0      0.129598   artist   Michael Angelo is the most famous artist.  
1      0.083758     star     Michael Angelo is the most famous star.  
2      0.071271  pianist  Michael Angelo is the most famous pianist.  
3      0.068525    actor    Michael Angelo is the most famous actor.  
4      0.041793      man      Michael Angelo is the most famous man.

## Miejscownik - o kim? o czym?

In [46]:
test_models("Nowa antena została zamontowana na [MASK].")

model      bert                                               distilbert  \
field     score token                                sentence      score   
0      0.051040    pl  Nowa antena została zamontowana na pl.   0.084255   
1      0.034817     s   Nowa antena została zamontowana na s.   0.060161   
2      0.028756    pt  Nowa antena została zamontowana na pt.   0.038168   
3      0.025073     p   Nowa antena została zamontowana na p.   0.025246   
4      0.025065    ul  Nowa antena została zamontowana na ul.   0.022759   

model                                                           roberta  \
field     token                                      sentence     score   
0        stacji    Nowa antena została zamontowana na stacji.  0.131744   
1          nowy      Nowa antena została zamontowana na nowy.  0.118101   
2             ң         Nowa antena została zamontowana na ң.  0.086710   
3       miejscu   Nowa antena została zamontowana na miejscu.  0.038242   
4      południe  Nowa antena została zamontowana na południe.  0.032310   

model                                                          
field     token                                      sentence  
0       piętrze   Nowa antena została zamontowana na piętrze.  
1       budynku   Nowa antena została zamontowana na budynku.  
2      zewnątrz  Nowa antena została zamontowana na zewnątrz.  
3         rynku     Nowa antena została zamontowana na rynku.  
4       miejscu   Nowa antena została zamontowana na miejscu.

In [47]:
test_models("New antenna was mounted on [MASK]")

model      bert                                       distilbert           \
field     score token                        sentence      score    token   
0      0.654258     .     New antenna was mounted on.   0.144328        :   
1      0.106929     ;    New antenna was mounted on ;   0.041206  display   
2      0.049844     :    New antenna was mounted on :   0.035538        .   
3      0.043600   the  New antenna was mounted on the   0.025901    board   
4      0.014715     a    New antenna was mounted on a   0.015708        ;   

model                                       roberta        \
field                            sentence     score token   
0            New antenna was mounted on :  0.150708   ...   
1      New antenna was mounted on display  0.141113  </s>   
2             New antenna was mounted on.  0.138151   ...   
3        New antenna was mounted on board  0.039595     .   
4            New antenna was mounted on ;  0.032985  wall   

model                                   
field                         sentence  
0        New antenna was mounted on...  
1           New antenna was mounted on  
2        New antenna was mounted on...  
3          New antenna was mounted on.  
4      New antenna was mounted on wall

## Devise a method to test long-range relationships such as gender

In [30]:
test_models("Ona jest bardzo [MASK] kobietą.")

model      bert                                          distilbert  \
field     score   token                         sentence      score   
0      0.120775    dużą    Ona jest bardzo dużą kobietą.   0.364548   
1      0.084604  dobrze  Ona jest bardzo dobrze kobietą.   0.125895   
2      0.041336   dobra   Ona jest bardzo dobra kobietą.   0.023124   
3      0.041109       z       Ona jest bardzo z kobietą.   0.022898   
4      0.032673  bardzo  Ona jest bardzo bardzo kobietą.   0.019438   

model                                                  roberta          \
field      token                            sentence     score   token   
0          znana      Ona jest bardzo znana kobietą.  0.695047   dobrą   
1         bardzo     Ona jest bardzo bardzo kobietą.  0.073055  dobrym   
2              익          Ona jest bardzo 익 kobietą.  0.072841    sexy   
3           sama       Ona jest bardzo sama kobietą.  0.020107   dobra   
4      wszystkim  Ona jest bardzo wszystkim kobietą.  0.017573  piękna   

model                                   
field                         sentence  
0       Ona jest bardzo dobrą kobietą.  
1      Ona jest bardzo dobrym kobietą.  
2        Ona jest bardzo sexy kobietą.  
3       Ona jest bardzo dobra kobietą.  
4      Ona jest bardzo piękna kobietą.

In [33]:
test_models("Tomek poszukuje [MASK] pracowniczki do roboty.")

model      bert                                                          \
field     score   token                                        sentence   
0      0.280238     jej     Tomek poszukuje jej pracowniczki do roboty.   
1      0.145836   swoją   Tomek poszukuje swoją pracowniczki do roboty.   
2      0.066518  swojej  Tomek poszukuje swojej pracowniczki do roboty.   
3      0.044535    jego    Tomek poszukuje jego pracowniczki do roboty.   
4      0.043566     się     Tomek poszukuje się pracowniczki do roboty.   

model distilbert             \
field      score      token   
0       0.042825      prace   
1       0.036200     swoich   
2       0.027344  wszystkie   
3       0.024547  działania   
4       0.023663       jego   

model                                                      roberta           \
field                                           sentence     score    token   
0          Tomek poszukuje prace pracowniczki do roboty.  0.542353   swojej   
1         Tomek poszukuje swoich pracowniczki do roboty.  0.092808    nowej   
2      Tomek poszukuje wszystkie pracowniczki do roboty.  0.070701  własnej   
3      Tomek poszukuje działania pracowniczki do roboty.  0.031823    pracy   
4           Tomek poszukuje jego pracowniczki do roboty.  0.024536  drugiej   

model                                                   
field                                         sentence  
0       Tomek poszukuje swojej pracowniczki do roboty.  
1        Tomek poszukuje nowej pracowniczki do roboty.  
2      Tomek poszukuje własnej pracowniczki do roboty.  
3        Tomek poszukuje pracy pracowniczki do roboty.  
4      Tomek poszukuje drugiej pracowniczki do roboty.

In [32]:
test_models("Poszła do szkoły i [MASK] jedynkę z matmy.")

model      bert                                                      \
field     score   token                                    sentence   
0      0.173613      na      Poszła do szkoły i na jedynkę z matmy.   
1      0.088003       w       Poszła do szkoły i w jedynkę z matmy.   
2      0.066776      do      Poszła do szkoły i do jedynkę z matmy.   
3      0.044038      to      Poszła do szkoły i to jedynkę z matmy.   
4      0.043195  zajęła  Poszła do szkoły i zajęła jedynkę z matmy.   

model distilbert                                                            \
field      score      token                                       sentence   
0       0.408028      miała      Poszła do szkoły i miała jedynkę z matmy.   
1       0.164880          辆          Poszła do szkoły i 辆 jedynkę z matmy.   
2       0.085786       miał       Poszła do szkoły i miał jedynkę z matmy.   
3       0.045111         ma         Poszła do szkoły i ma jedynkę z matmy.   
4       0.015711  prowadził  Poszła do szkoły i prowadził jedynkę z matmy.   

model   roberta                                                        
field     score    token                                     sentence  
0      0.295932    miała    Poszła do szkoły i miała jedynkę z matmy.  
1      0.122470       ma       Poszła do szkoły i ma jedynkę z matmy.  
2      0.118518     dala     Poszła do szkoły i dala jedynkę z matmy.  
3      0.058293       na       Poszła do szkoły i na jedynkę z matmy.  
4      0.032068  została  Poszła do szkoły i została jedynkę z matmy.

## Check if the model captures real-world knolwedge

In [36]:
test_models("Planeta obraca się wokół [MASK].")

model      bert                                             distilbert  \
field     score    token                           sentence      score   
0      0.159912    Ziemi    Planeta obraca się wokół Ziemi.   0.529004   
1      0.103407   planet   Planeta obraca się wokół planet.   0.046062   
2      0.023412      tzw      Planeta obraca się wokół tzw.   0.026346   
3      0.019757        S        Planeta obraca się wokół S.   0.013868   
4      0.019730  planeta  Planeta obraca się wokół planeta.   0.013124   

model                                             roberta             \
field   token                          sentence     score      token   
0       Ziemi   Planeta obraca się wokół Ziemi.  0.294103        nas   
1      Europy  Planeta obraca się wokół Europy.  0.175451     siebie   
2       ziemi   Planeta obraca się wokół ziemi.  0.052579     Ciebie   
3      miasta  Planeta obraca się wokół miasta.  0.044376  człowieka   
4      siebie  Planeta obraca się wokół siebie.  0.025861     gwiazd   

model                                       
field                             sentence  
0            Planeta obraca się wokół nas.  
1         Planeta obraca się wokół siebie.  
2         Planeta obraca się wokół Ciebie.  
3      Planeta obraca się wokół człowieka.  
4         Planeta obraca się wokół gwiazd.

In [39]:
test_models("Liść ma kolor [MASK].")

model      bert                                  distilbert          \
field     score    token                sentence      score   token   
0      0.070845        K        Liść ma kolor K.   0.061500       瑕   
1      0.030189  brązowy  Liść ma kolor brązowy.   0.034252       駕   
2      0.024832       mm       Liść ma kolor mm.   0.023154   ciała   
3      0.020602        p        Liść ma kolor p.   0.011369    ##wy   
4      0.016935        k        Liść ma kolor k.   0.011294  polski   

model                          roberta                                       
field               sentence     score      token                  sentence  
0           Liść ma kolor 瑕.  0.297481    zielony    Liść ma kolor zielony.  
1           Liść ma kolor 駕.  0.178336   czerwony   Liść ma kolor czerwony.  
2       Liść ma kolor ciała.  0.126440      biały      Liść ma kolor biały.  
3           Liść ma kolorwy.  0.122614     czarny     Liść ma kolor czarny.  
4      Liść ma kolor polski.  0.122500  niebieski  Liść ma kolor niebieski.

## Answer the following questions:
 - Which of the models produced the best results?

Roberta was the model that was producing the most satisfying examples. 

 - Was any of the models able to capture Polish grammar?

Roberta was able to catch polish grammar like gender relations.

 - Was any of the models able to capture long-distant relationships between the words?

Roberta was able to capture correct examples for the second sentence.

 - Was any of the models able to capture world knowledge?

Distilbert and Roberta were able to capture a real world knowledge.

 - What are the most striking errors made by the models?

 Foreign words, non-latin alphabets, abreviations instead of full words, dots and other non-alphanumeric characters.

 